# Correlations and external validity (3)
Date: August 26th, 2023

Use source datasets to compute daily aggregates of the counts of trucks (monthly sums and daily averages) and export the results to excel for comparison with data from monthly report on border performances (November 2022 and December 2022 \*maybe January 2023, requires different merge strategy)

### Content:

1. [Import, process and clean toll road dataset](#1)
2. [Import and process crickmay dataset on time of counts of border crossings](#2)
3. [Merged table for truck counts only by day and month](#3)
4. [Correlations and cleaning of result table + export to excel](#4)
5. [Export of selected monthly aggregates: Nov 2022 and Dec 2022](#5)
6. [Miscellaneous: Notes on Little's Law](#6)


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import scipy as sp
#sns.set_theme(style="ticks")
plt.style.use('bmh')

<a id="1"></a>

## Import, process and clean toll road dataset
Standard procedure for 

In [2]:
def process_flows(flow):

    newcols = [x[0] for x in flow.columns][:13] + [",".join(x) for x in flow.columns][13:]
    flow.columns=newcols
    
    flow['year'] = flow['Date'].dt.year
    flow['month'] = flow['Date'].dt.month
    flow['YMD'] = flow['Date'].dt.year.astype(str) +"-"+flow['Date'].dt.month.astype(str).str.zfill(2)+"-"+flow['Date'].dt.day.astype(str)  
    return flow
 
#     flow['Dir 1'] = flow['Direction 1'].map({"To Beit Bridge":"SA-Zimbabwe","To Musina":"Zimbabwe-SA"})
#     flow['Dir 2'] = flow['Direction 2'].map({"To Beit Bridge":"SA-Zimbabwe","To Musina":"Zimbabwe-SA"})
#     return flow.copy()

flow3201= pd.read_excel("C:\\Users\\wb553912\\OneDrive - WBG\\Desktop\\RemSen\\ReferenceData\\CM_3201_Three HourlyFlow_Ext LHV.xlsx",header=[0,1])
flow3202= pd.read_excel("C:\\Users\\wb553912\\OneDrive - WBG\\Desktop\\RemSen\\ReferenceData\\CM_3202_Three HourlyFlow_Ext LHV.xlsx",header=[0,1])

flow3201 = process_flows(flow3201)
flow3202 = process_flows(flow3202)

In [3]:
flow3201

,Site ID,Site Name,Route,Total Directions,No,Direction 1,Dir 1 Description,No,Direction 2,Dir 2 Description,...,"Dir 2,Medium HV","Dir 2,Long HV","Road,Total","Road,Light","Road,Short HV","Road,Medium HV","Road,Long HV",year,month,YMD
0,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,12.0,18.0,489.0,405.0,29.0,25.0,30.0,2018,1,2018-01-1
1,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,10.0,38.0,471.0,380.0,12.0,26.0,53.0,2018,1,2018-01-1
2,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,7.0,14.0,462.0,419.0,9.0,9.0,25.0,2018,1,2018-01-1
3,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,1.0,19.0,486.0,422.0,16.0,10.0,38.0,2018,1,2018-01-1
4,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,6.0,12.0,608.0,554.0,12.0,20.0,22.0,2018,1,2018-01-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14603,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,12.0,31.0,988.0,855.0,47.0,23.0,63.0,2022,12,2022-12-31
14604,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,18.0,41.0,1112.0,957.0,54.0,33.0,68.0,2022,12,2022-12-31
14605,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,10.0,66.0,1031.0,859.0,51.0,20.0,101.0,2022,12,2022-12-31
14606,3201,Beit Bridge SB,N001,2,1,To Musina,North,2,To Beit Bridge,South,...,8.0,47.0,709.0,569.0,35.0,27.0,78.0,2022,12,2022-12-31


In [4]:
assert len(flow3201['Direction 1'].unique())==len(flow3201['Direction 2'].unique()) and len(flow3201['Direction 2'].unique())==1

mydict3201 = {"Dir 1":flow3201['Direction 1'].values[0],"Dir 2":flow3201['Direction 2'].values[0],'Road':"Road"}

In [5]:
assert len(flow3202['Direction 1'].unique())==len(flow3202['Direction 2'].unique()) and len(flow3202['Direction 2'].unique())==1

mydict3202 = {"Dir 1":flow3202['Direction 1'].values[0],"Dir 2":flow3202['Direction 2'].values[0],'Road':"Road"}

In [6]:
print(mydict3201)
print(mydict3202)

{'Dir 1': 'To Musina', 'Dir 2': 'To Beit Bridge', 'Road': 'Road'}
{'Dir 1': 'To Beit Bridge', 'Dir 2': 'To Musina', 'Road': 'Road'}


In [7]:
def adj_named_columns(fl, mydict):
    newcols = []
    print("Old column names: ",fl.columns)
    for e in fl.columns:
            sp = e.split(",")
            if len(sp)>1:
                print(sp[1])
                try:
                    sp2 = mydict.get(sp[0])+","+sp[1]
                    newcols.append(sp2)
                except TypeError as tp:
                    print("issue",sp[0],mydict.get(sp[0]))
                    print(tp)
            else:
                newcols.append(e)
            
    print("New column names: ", newcols)
    return newcols

print("Location: 3201\n*****************")
c3201 = adj_named_columns(flow3201, mydict3201)
      
print("Location: 3202\n*****************")
c3202 = adj_named_columns(flow3202, mydict3202)

flow3201.columns=c3201
flow3202.columns=c3202

Location: 3201
*****************
Old column names:  Index(['Site ID', 'Site Name', 'Route', 'Total Directions', 'No',
       'Direction 1', 'Dir 1 Description', 'No', 'Direction 2',
       'Dir 2 Description', 'Date', 'Period End', 'Duration', 'Dir 1,Total',
       'Dir 1,Light', 'Dir 1,Short HV', 'Dir 1,Medium HV', 'Dir 1,Long HV',
       'Dir 2,Total', 'Dir 2,Light', 'Dir 2,Short HV', 'Dir 2,Medium HV',
       'Dir 2,Long HV', 'Road,Total', 'Road,Light', 'Road,Short HV',
       'Road,Medium HV', 'Road,Long HV', 'year', 'month', 'YMD'],
      dtype='object')
Total
Light
Short HV
Medium HV
Long HV
Total
Light
Short HV
Medium HV
Long HV
Total
Light
Short HV
Medium HV
Long HV
New column names:  ['Site ID', 'Site Name', 'Route', 'Total Directions', 'No', 'Direction 1', 'Dir 1 Description', 'No', 'Direction 2', 'Dir 2 Description', 'Date', 'Period End', 'Duration', 'To Musina,Total', 'To Musina,Light', 'To Musina,Short HV', 'To Musina,Medium HV', 'To Musina,Long HV', 'To Beit Bridge,Total'

In [8]:
print(flow3201)
print(flow3202)

       Site ID       Site Name Route  Total Directions  No Direction 1  \
0         3201  Beit Bridge SB  N001                 2   1   To Musina   
1         3201  Beit Bridge SB  N001                 2   1   To Musina   
2         3201  Beit Bridge SB  N001                 2   1   To Musina   
3         3201  Beit Bridge SB  N001                 2   1   To Musina   
4         3201  Beit Bridge SB  N001                 2   1   To Musina   
...        ...             ...   ...               ...  ..         ...   
14603     3201  Beit Bridge SB  N001                 2   1   To Musina   
14604     3201  Beit Bridge SB  N001                 2   1   To Musina   
14605     3201  Beit Bridge SB  N001                 2   1   To Musina   
14606     3201  Beit Bridge SB  N001                 2   1   To Musina   
14607     3201  Beit Bridge SB  N001                 2   1   To Musina   

      Dir 1 Description  No     Direction 2 Dir 2 Description  ...  \
0                 North   2  To Beit Brid

In [9]:
flow3202 = flow3202[['Site ID', 'Site Name', 'Route', 'Total Directions', 
       'Direction 1', 'Dir 1 Description', 'Direction 2',
       'Dir 2 Description', 'Date', 'Period End', 'Duration',
       'To Beit Bridge,Total', 'To Beit Bridge,Light',
       'To Beit Bridge,Short HV', 'To Beit Bridge,Medium HV',
       'To Beit Bridge,Long HV', 'To Musina,Total', 'To Musina,Light',
       'To Musina,Short HV', 'To Musina,Medium HV', 'To Musina,Long HV',
       'Road,Total', 'Road,Light', 'Road,Short HV', 'Road,Medium HV',
       'Road,Long HV', 'year', 'month', 'YMD']]

flow3201 = flow3201[['Site ID', 'Site Name', 'Route', 'Total Directions',
       'Direction 1', 'Dir 1 Description','Direction 2',
       'Dir 2 Description', 'Date', 'Period End', 'Duration',
       'To Musina,Total', 'To Musina,Light', 'To Musina,Short HV',
       'To Musina,Medium HV', 'To Musina,Long HV', 'To Beit Bridge,Total',
       'To Beit Bridge,Light', 'To Beit Bridge,Short HV',
       'To Beit Bridge,Medium HV', 'To Beit Bridge,Long HV', 'Road,Total',
       'Road,Light', 'Road,Short HV', 'Road,Medium HV', 'Road,Long HV', 'year',
       'month', 'YMD']]

fl1 = flow3201.drop(columns=['Site ID', 'Site Name', 'Route', 'Total Directions','Direction 1', 'Dir 1 Description','Direction 2','Dir 2 Description', 'Date', 'Period End', 'Duration']).groupby(['YMD','year','month']).sum()
fl2 = flow3202.drop(columns=['Site ID', 'Site Name', 'Route', 'Total Directions','Direction 1', 'Dir 1 Description','Direction 2','Dir 2 Description', 'Date', 'Period End', 'Duration']).groupby(['YMD','year','month']).sum()

In [10]:
fl1.columns = [x+"_3201" for x in fl1.columns]
fl2.columns = [x+"_3202" for x in fl2.columns]
flow = fl1.join(fl2)
flow = flow.reset_index()
flow['YMD'] = pd.to_datetime(flow['YMD'])

 <a id="2"></a>
## Import and process crickmay dataset on time of counts of border crossings

### Crickmay border crossings and time aggregates

In [11]:
bb= pd.read_csv("C:\\Users\\wb553912\\OneDrive - WBG\\Desktop\\RemSen\\ReferenceData\\Beitbridge_Border_2017_02_01_to_2023_02_28.csv").dropna(subset='ProjectID')

In [12]:
bb

,ProjectID,StartDate,StartHour,Direction,GeozoneName,IsGrandTotalRowTotal,Border_Crossing_Count_Events,Border_Crossing_Median_Minutes,Border_Crossing_Bottom_10__Minutes,Border_Crossing_Top_10__Minutes
1,1.0,2017-02-01 00:00:00.000,0.0,SA-Zimbabwe,Beitbridge Border,False,22,1664.500000,683.400000,6648.900000
2,1.0,2017-02-01 00:00:00.000,0.0,Zimbabwe-SA,Beitbridge Border,False,6,613.000000,324.500000,615.000000
3,1.0,2017-02-01 00:00:00.000,1.0,SA-Zimbabwe,Beitbridge Border,False,3,3349.000000,1426.600000,9247.400000
4,1.0,2017-02-01 00:00:00.000,1.0,Zimbabwe-SA,Beitbridge Border,False,1,204.000000,204.000000,204.000000
5,1.0,2017-02-01 00:00:00.000,2.0,SA-Zimbabwe,Beitbridge Border,False,3,3355.000000,2568.600000,6473.400000
...,...,...,...,...,...,...,...,...,...,...
68314,3.0,2023-02-28 00:00:00.000,18.0,Zimbabwe- SA,Beitbridge Border,False,1,888.566666,888.566666,888.566666
68315,3.0,2023-02-28 00:00:00.000,19.0,SA- Zimbabwe,Beitbridge Border,False,4,2092.491666,1445.743333,3045.151666
68316,3.0,2023-02-28 00:00:00.000,19.0,Zimbabwe- SA,Beitbridge Border,False,1,3773.300000,3773.300000,3773.300000
68317,3.0,2023-02-28 00:00:00.000,20.0,Zimbabwe- SA,Beitbridge Border,False,1,441.833333,441.833333,441.833333


In [14]:
bb= bb.dropna(subset=['StartHour'])
bb['Direction'] = bb['Direction'].apply(lambda x: str(x).strip())
bb['Direction'] = bb['Direction'].apply(lambda x: str(x).replace(" ",''))
bb= bb.copy()
bb['date'] = pd.to_datetime(bb['StartDate'])
daily_bb = bb.groupby(['date','Direction'])['Border_Crossing_Count_Events'].sum().unstack(level=1)
daily_bb.reset_index().dtypes

Direction
date           datetime64[ns]
SA-Zimbabwe           float64
Zimbabwe-SA           float64
dtype: object

 <a id="3"></a>
## Merged table

### Merge both data sources, correct column names, deduplicate correlation values and save to excel

In [15]:
out = flow.set_index('YMD').join(daily_bb)
out = out.dropna()
#out.drop(columns=['index'],inplace=True)

 <a id="4"></a>
## Correlations and cleaning of result table + export to excel

In [16]:
cor = out.drop(columns=['month','year']).corr().stack().reset_index()
cor = cor[cor['level_0'].isin(['Zimbabwe-SA','SA-Zimbabwe'])]

In [17]:
cor

,level_0,level_1,0
960,SA-Zimbabwe,"To Musina,Total_3201",0.411374
961,SA-Zimbabwe,"To Musina,Light_3201",0.391005
962,SA-Zimbabwe,"To Musina,Short HV_3201",-0.081600
963,SA-Zimbabwe,"To Musina,Medium HV_3201",0.538633
964,SA-Zimbabwe,"To Musina,Long HV_3201",0.417880
...,...,...,...
1019,Zimbabwe-SA,"Road,Short HV_3202",0.222556
1020,Zimbabwe-SA,"Road,Medium HV_3202",0.390785
1021,Zimbabwe-SA,"Road,Long HV_3202",0.112332
1022,Zimbabwe-SA,SA-Zimbabwe,0.678275


<a id="5"></a> 
## Extract Nov, Dec 2022 and Jan 2023 values for validation and comparison with PDF report values 

In [18]:
nov22 = out[(out['year']==2022)&(out['month']==11)]
nov22_avg = nov22.drop(columns=['month','year']).mean()
nov22_sum = nov22.drop(columns=['month','year']).sum()
nov22_avg.to_csv("Nov 2022 merged.csv")
nov22_sum

To Musina,Total_3201              85840.0
To Musina,Light_3201              69553.0
To Musina,Short HV_3201            3984.0
To Musina,Medium HV_3201           3552.0
To Musina,Long HV_3201             8751.0
To Beit Bridge,Total_3201         85278.0
To Beit Bridge,Light_3201         69708.0
To Beit Bridge,Short HV_3201       2434.0
To Beit Bridge,Medium HV_3201      2971.0
To Beit Bridge,Long HV_3201       10165.0
Road,Total_3201                  171118.0
Road,Light_3201                  139261.0
Road,Short HV_3201                 6418.0
Road,Medium HV_3201                6523.0
Road,Long HV_3201                 18916.0
To Beit Bridge,Total_3202         88312.0
To Beit Bridge,Light_3202         66625.0
To Beit Bridge,Short HV_3202       6647.0
To Beit Bridge,Medium HV_3202      4110.0
To Beit Bridge,Long HV_3202       10930.0
To Musina,Total_3202              86189.0
To Musina,Light_3202              63275.0
To Musina,Short HV_3202            6676.0
To Musina,Medium HV_3202          

In [19]:
dec22 = out[(out['year']==2022)&(out['month']==12)]
dec22_avg = dec22.drop(columns=['month','year']).mean()
dec22_sum = dec22.drop(columns=['month','year']).sum()
dec22_avg.to_csv("Dec 2022 merged.csv")

In [20]:
jan22 = out[(out['year']==2023)&(out['month']==1)]
jan22_avg = jan22.drop(columns=['month','year']).mean()
jan22_sum =jan22.drop(columns=['month','year']).sum()
jan22_avg.to_csv("Jan 2023 merged.csv")

--------------------------------------------

<a id="6"></a>

## Miscellaneous: Notes on Little's Law

### Little’s Law

__L__ – the average number of items in a queuing system

__$\lambda$__ – the average number of items arriving at the system per unit of time

__W__ – the average waiting time an item spends in a queuing system

 
##### Example of Little’s Law

We want to know the average number of trucks queuing at the border crossing facility (to decide whether we needs to add more space to accommodate more trucks). Currently, the queuing area can accommodate no more than X trucks (???)

Measure on average, X trucks arrive at the border every hour. Determine average time spent at the border.  Given these inputs, we can find the average number of trucks queuing in at the border by applying Little’s Law:

##### $L$  =  $X$ trucks/hour x $Y$ hour  = N trucks
 
Little’s Law shows that, on average, there are only N trucks queuing at the border.


#### Extension

>Note ON ASUMTIONS:  When mathematically applying Little’s law the pre-condition is that the variables are independent, the assumption is that number of ITEMS in the queue does not impact on how quickly the SYSTEM works. (??? it does tho)

>The principle is that __WiP (work in progress)__ and __Throughput__ are two independent variables and changing one does not change the other, only __cycle time__ is impacted.


__$\lambda$__ is Throughput (how many items you complete for a certain period of time) instead of customers’ arrival rate.

__$W$__ is Cycle Time (the elapsed time an item spends to go through your workflow) instead of the time people spend at a business.

__$L$__ is Work in Progress (or how many items stay in your system at any one time) instead of customer volume.

So $L = \lambda * W$ becomes 

$$WIP = Throughput * Cycle Time$$
